Implementation of 2D elasto-plastic problem using FEniCS-X. The code is based on a [legacy solution](https://comet-fenics.readthedocs.io/en/latest/demo/2D_plasticity/vonMises_plasticity.py.html) for FEniCS 2019.

In [2]:
import meshio
import numpy as np

import ufl
from dolfinx import fem, io, plot

from mpi4py import MPI
from petsc4py.PETSc import ScalarType
# import pyvista
from petsc4py import PETSc

In [3]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [4]:
#It works with the msh4 only!!
msh = meshio.read("thick_cylinder.msh")

# Create and save one file for the mesh, and one file for the facets 
triangle_mesh = create_mesh(msh, "triangle", prune_z=True)
line_mesh = create_mesh(msh, "line", prune_z=True)
meshio.write("thick_cylinder.xdmf", triangle_mesh)
meshio.write("mt.xdmf", line_mesh)

with io.XDMFFile(MPI.COMM_WORLD, "thick_cylinder.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    ct = xdmf.read_meshtags(mesh, name="Grid")

mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)

with io.XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    ft = xdmf.read_meshtags(mesh, name="Grid")

In [5]:
msh

<meshio mesh object>
  Number of points: 811
  Number of cells:
    line: 10
    line: 69
    line: 10
    line: 53
    triangle: 1478
  Cell sets: gmsh:bounding_entities
  Point data: gmsh:dim_tags
  Cell data: gmsh:physical, gmsh:geometrical

In [6]:
mesh

In [7]:
# elastic parameters
E = 70e3
nu = 0.3
lmbda = E*nu/(1+nu)/(1-2*nu)
mu = fem.Constant(mesh, ScalarType(E/2./(1+nu)))
sig0 = fem.Constant(mesh, ScalarType(250))  # yield strength
Et = E/100.  # tangent modulus
H = E*Et/(E-Et)  # hardening modulus

Re, Ri = 1.3, 1.   # external/internal radius
ds = ufl.Measure("ds", domain=mesh)

In [8]:
deg_u = 2
deg_stress = 2
V = fem.VectorFunctionSpace(mesh, ("CG", deg_u))
We = ufl.VectorElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, dim=4, quad_scheme='default')
W0e = ufl.FiniteElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, quad_scheme='default')
W = fem.FunctionSpace(mesh, We)
W0 = fem.FunctionSpace(mesh, W0e)

In [9]:
sig = fem.Function(W)
sig_old = fem.Function(W)
n_elas = fem.Function(W)
beta = fem.Function(W0)
p = fem.Function(W0)# , name="Cumulative plastic strain"
dp = fem.Function(W0)
u = fem.Function(V)#, name="Total displacement"
du = fem.Function(V)#, name="Iteration correction"
Du = fem.Function(V)# , name="Current increment"
v = ufl.TrialFunction(V)
u_ = ufl.TestFunction(V)

zero_Du = fem.Function(V)


In [10]:
left_marker = 3
down_marker = 1
left_facets = ft.indices[ft.values == left_marker]
down_facets = ft.indices[ft.values == down_marker]
left_dofs = fem.locate_dofs_topological(V.sub(0), mesh.topology.dim-1, left_facets)
down_dofs = fem.locate_dofs_topological(V.sub(1), mesh.topology.dim-1, down_facets)

bcs = [fem.dirichletbc(ScalarType(0), left_dofs, V.sub(0)), fem.dirichletbc(ScalarType(0), down_dofs, V.sub(1))]

In [23]:
n = ufl.FacetNormal(mesh)
q_lim = float(2/np.sqrt(3)*np.log(Re/Ri)*sig0.value)

# class Loading:
#     def __init__(self):
#         self.q = q_lim
#         self.t = 0.0

#     def eval(self, x):
#         return np.full(x.shape[1], -self.q * self.t)

# loading = Loading()
# load_func.interpolate(loading.eval)
V_real = fem.FunctionSpace(mesh, ("CG", deg_u))
loading = fem.Function(V_real)

loading.interpolate(lambda x: (np.zeros_like(x[1])))
loading.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

def F_ext(v):
    return -loading * ufl.inner(n, v)*ds(4)

In [12]:
def eps(v):
    e = ufl.sym(ufl.grad(v))
    return ufl.as_tensor([[e[0, 0], e[0, 1], 0],
                          [e[0, 1], e[1, 1], 0],
                          [0, 0, 0]])
def sigma(eps_el):
    return lmbda*ufl.tr(eps_el)*ufl.Identity(3) + 2*mu*eps_el

def as_3D_tensor(X):
    return ufl.as_tensor([[X[0], X[3], 0],
                          [X[3], X[1], 0],
                          [0, 0, X[2]]])

In [13]:
ppos = lambda x: (x + ufl.sqrt(x**2))/2.
def proj_sig(deps, old_sig, old_p):
    sig_n = as_3D_tensor(old_sig)
    sig_elas = sig_n + sigma(deps)
    s = ufl.dev(sig_elas)
    sig_eq = ufl.sqrt(3/2.*ufl.inner(s, s))
    f_elas = sig_eq - sig0 - H*old_p
    # print(sig_eq)
    dp = ppos(f_elas)/(3*mu+H)
    n_elas = s/sig_eq*ppos(f_elas)/f_elas
    beta = 3*mu*dp/sig_eq
    new_sig = sig_elas-beta*s
    # new_sig = old_p*s
    return ufl.as_vector([new_sig[0, 0], new_sig[1, 1], new_sig[2, 2], new_sig[0, 1]]), \
           ufl.as_vector([n_elas[0, 0], n_elas[1, 1], n_elas[2, 2], n_elas[0, 1]]), \
           beta, dp

In [14]:
def sigma_tang(e):
    N_elas = as_3D_tensor(n_elas)
    # return sigma(e) - 3*mu*(3*mu/(3*mu+H)-beta)*inner(N_elas, e)*N_elas 
    return sigma(e) - 3*mu*(3*mu/(3*mu+H)-beta)*ufl.inner(N_elas, e)*N_elas - 2*mu*beta*ufl.dev(e)

In [24]:
dx = ufl.Measure(
    "dx",
    domain=mesh,
    metadata={"quadrature_degree": deg_stress, "quadrature_scheme": "default"},
)

a_Newton = ufl.inner(eps(v), sigma_tang(eps(u_)))*dx
res = ufl.inner(eps(u_), as_3D_tensor(sig))*dx + F_ext(u_)

In [16]:
# def local_project(v, V, u=None):
#     dv = ufl.TrialFunction(V)
#     v_ = ufl.TestFunction(V)
#     a_proj = inner(dv, v_)*dxm
#     b_proj = inner(v, v_)*dxm
#     solver = LocalSolver(a_proj, b_proj)
#     solver.factorize()
#     if u is None:
#         u = fem.Function(V)
#         solver.solve_local_rhs(u)
#         return u
#     else:
#         solver.solve_local_rhs(u)
#         return

def project(expression, V):
    u_, v_ = ufl.TrialFunction(V), ufl.TestFunction(V)
    a_p = ufl.inner(u_, v_) * dx
    L_p = ufl.inner(expression, v_) * dx
    projection = fem.petsc.LinearProblem(a_p, L_p) #dolfinx.fem.LinearProblem(a, L, [bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"}) #dolfinx.fem.LinearProblem(a, L, bcs=[], u=None, petsc_options={},form_compiler_parameters={"ksp_type": "preonly", "pc_type": "lu"},jit_parameters={})
    return projection.solve()

def project(v, target_func, bcs=[]):
    # v->target_func
    # Ensure we have a mesh and attach to measure
    V = target_func.function_space
    # dx = ufl.dx(V.mesh)

    # Define variational problem for projection
    w = ufl.TestFunction(V)
    Pv = ufl.TrialFunction(V)
    a = fem.form(ufl.inner(Pv, w) * dx)
    L = fem.form(ufl.inner(v, w) * dx)

    # Assemble linear system
    A = fem.petsc.assemble_matrix(a, bcs)
    A.assemble()
    b = fem.petsc.assemble_vector(L)
    fem.petsc.apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, bcs)

    # Solve linear system
    solver = PETSc.KSP().create(A.getComm())
    solver.setOperators(A)
    solver.solve(b, target_func.vector)

In [17]:
u_test = fem.Function(V)
eps_test = eps(u_test)
sigma_0 = sigma(eps_test)
sigma_test = fem.Function(W)
beta_test = fem.Function(W0)
project(ufl.as_vector([sigma_0[0, 0], sigma_0[1, 1], sigma_0[2,2], sigma_0[0, 1]])
, sigma_test)


In [18]:
sig_, _, _, _ = proj_sig(eps_test, sig_old, p)
project(sig_, sigma_test)

In [19]:
beta_test = fem.Function(W0)
project(p, beta_test)

In [20]:
P0 = fem.FunctionSpace(mesh, ("DG", 0))
p_avg = fem.Function(P0, name="Plastic strain")

In [25]:
A = fem.petsc.assemble_matrix(fem.form(a_Newton), bcs=bcs)
Res = fem.petsc.create_vector(fem.form(res))

solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

In [27]:
Res.array.shape

(6198,)

In [26]:
problem = fem.petsc.LinearProblem(
    a_Newton,
    res,
    bcs=bcs,
    petsc_options={
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
)

In [26]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0, 1.1, Nincr+1)[1:]**0.5
results = np.zeros((Nincr+1, 2))
for (i, t) in enumerate(load_steps):
    # loading.t = t
    # load_func.interpolate(loading.eval)
    loading.interpolate(lambda x: (t * q_lim * np.ones_like(x[1])))
    loading.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )

    fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
    A.assemble()

    with Res.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(Res, fem.form(res))
    Res.assemble()

    fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
    Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(Res, bcs)

    nRes0 = Res.norm() # Which one?
    nRes = 1
    zero_Du.vector.copy(Du.vector)
    Du.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    # Du.interpolate(lambda x : (np.zeros_like(x[0]), np.zeros_like(x[1])))
    # Du.interpolate(fem.Constant(mesh, (ScalarType(0), ScalarType(0))))
    print("Increment:", str(i+1), t * q_lim)
    niter = 0
     
    while nRes > tol and niter < Nitermax:
        solver.solve(Res, du.vector) 
        du.x.scatter_forward() 
        print(du.vector.max())

        Du.vector.axpy(1, du.vector) # Du = Du + 1*du
        Du.x.scatter_forward() 

        deps = eps(Du)
        sig_, n_elas_, beta_, dp_ = proj_sig(deps, sig_old, p)

        project(sig_, sig)
        project(n_elas_, n_elas)
        project(beta_, beta)
        
        fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
        A.assemble()
        fem.petsc.assemble_vector(Res, fem.form(res))
        # with Res.localForm() as loc_b:
        #     loc_b.set(0)
        Res.assemble()

        fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
        Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(Res, bcs)

        nRes = Res.norm()
        print("    Residual:", nRes, dp.vector.max())
        niter += 1
    u.vector.axpy(1, Du.vector) # u = u + 1*Du
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    sig.vector.copy(sig_old.vector)
    sig_old.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    project(dp_, dp)
    p.vector.axpy(1, dp.vector)
    p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)


Increment: 1 17.7621446711497
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 2 25.119465890772904
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 3 30.764937021820067
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 4 35.5242893422994
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 5 39.717362910876375
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 6 43.50819118181207
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 7 46.99421755101324
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 8 50.23893178154581
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 9 53.286434013449096
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 10 56.16883329025551
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 11 58.910369346214146
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 12 61.529874043640135
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 13 64.0423233740397
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 14 66.45985981375466
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 15 68.7924905042896
(0, 0.0)
    Residual: 

In [ ]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0, 1.1, Nincr+1)[1:]**0.5
results = np.zeros((Nincr+1, 2))
t = load_steps[0]
i = 0
for (i, t) in enumerate(load_steps):
    # loading.t = t
    # load_func.interpolate(loading.eval)
    loading.interpolate(lambda x: (t * q_lim * np.ones_like(x[1])))
    loading.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    # A, Res = assemble_system(a_Newton, res, bcs)
    fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
    A.assemble()
    # fem.petsc.assemble_vector(Res, fem.form(res))
    with Res.localForm() as loc_b:
        loc_b.set(0)
    # Res.assemble()
    fem.petsc.assemble_vector(Res, fem.form(res))

    fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
    Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(Res, bcs)
    # uh = fem.Function(V)
    print(Res.array.max())
    nRes0 = Res.norm() # Which one?
    nRes = 1
    # zero_Du.vector.copy(Du.vector)
    # Du.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    # Du.interpolate(lambda x : (np.zeros_like(x[0]), np.zeros_like(x[1])))
    # Du.interpolate(fem.Constant(mesh, (ScalarType(0), ScalarType(0))))
    print("Increment:", str(i+1), t * q_lim)
    niter = 0
        
    while nRes > tol and niter < Nitermax:
        # du = problem.solve()
        print(du.vector.max())
        solver.solve(Res, du.vector) #, "mumps"
        du.x.scatter_forward() 
        print(du.vector.min())

        # Du.assign(Du+du)
        Du.vector.axpy(1, du.vector) # Du = Du + 1*du
        Du.x.scatter_forward() 

        deps = eps(Du)
        sig_, n_elas_, beta_, dp_ = proj_sig(deps, sig_old, p)
        # local_project(sig_, W, sig)
        # local_project(n_elas_, W, n_elas)
        # local_project(beta_, W0, beta)
        # print(sig_, '\n', sig)
        project(sig_, sig)
        project(n_elas_, n_elas)
        project(beta_, beta)
        
        # A, Res = assemble_system(a_Newton, res, bcs)

        # fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
        # A.assemble()
        fem.petsc.assemble_vector(Res, fem.form(res))
        # with Res.localForm() as loc_b:
        #     loc_b.set(0)
        Res.assemble()

        fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
        Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(Res, bcs)

        nRes = Res.norm()
        print("    Residual:", nRes, dp.vector.max())
        niter += 1
    # u.assign(u+Du)
    u.vector.axpy(1, Du.vector) # u = u + 1*Du
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # sig_old.assign(sig)
    # p.assign(p+local_project(dp_, W0))
    sig.vector.copy(sig_old.vector)
    sig_old.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    project(dp_, dp)
    p.vector.axpy(1, dp.vector)
    p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)


In [24]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0, 1.1, Nincr+1)[1:]**0.5
results = np.zeros((Nincr+1, 2))
for (i, t) in enumerate(load_steps):
    # loading.t = t
    # load_func.interpolate(loading.eval)
    loading.interpolate(lambda x: (t * q_lim * np.ones_like(x[1])))
    loading.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    # A, Res = assemble_system(a_Newton, res, bcs)
    fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
    A.assemble()
    # fem.petsc.assemble_vector(Res, fem.form(res))
    with Res.localForm() as loc_b:
        loc_b.set(0)
    # Res.assemble()
    fem.petsc.assemble_vector(Res, fem.form(res))

    fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
    Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(Res, bcs)
    # uh = fem.Function(V)
    nRes0 = Res.norm() # Which one?
    nRes = 1
    # zero_Du.vector.copy(Du.vector)
    # Du.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    # Du.interpolate(lambda x : (np.zeros_like(x[0]), np.zeros_like(x[1])))
    # Du.interpolate(fem.Constant(mesh, (ScalarType(0), ScalarType(0))))
    print("Increment:", str(i+1), t * q_lim)
    niter = 0
     
    while nRes > tol and niter < Nitermax:
        # du = problem.solve()
        print(du.vector.max())
        solver.solve(Res, du.vector) #, "mumps"
        du.x.scatter_forward() 
        print(du.vector.min())

        # Du.assign(Du+du)
        Du.vector.axpy(1, du.vector) # Du = Du + 1*du
        Du.x.scatter_forward() 

        deps = eps(Du)
        sig_, n_elas_, beta_, dp_ = proj_sig(deps, sig_old, p)
        # local_project(sig_, W, sig)
        # local_project(n_elas_, W, n_elas)
        # local_project(beta_, W0, beta)
        # print(sig_, '\n', sig)
        project(sig_, sig)
        project(n_elas_, n_elas)
        project(beta_, beta)
        
        # A, Res = assemble_system(a_Newton, res, bcs)

        # fem.petsc.assemble_matrix(A, fem.form(a_Newton), bcs=bcs)
        # A.assemble()
        fem.petsc.assemble_vector(Res, fem.form(res))
        # with Res.localForm() as loc_b:
        #     loc_b.set(0)
        Res.assemble()

        fem.petsc.apply_lifting(Res, [fem.form(a_Newton)], [bcs])
        Res.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(Res, bcs)

        nRes = Res.norm()
        print("    Residual:", nRes, dp.vector.max())
        niter += 1
    # u.assign(u+Du)
    u.vector.axpy(1, Du.vector) # u = u + 1*Du
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

    # sig_old.assign(sig)
    # p.assign(p+local_project(dp_, W0))
    sig.vector.copy(sig_old.vector)
    sig_old.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    project(dp_, dp)
    p.vector.axpy(1, dp.vector)
    p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)


Increment: 1 25.119465890772904
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 2 35.5242893422994
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 3 43.50819118181207
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 4 50.23893178154581
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 5 56.16883329025551
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 6 61.529874043640135
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 7 66.45985981375466
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 8 71.0485786845988
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 9 75.3583976723187
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)
Increment: 10 79.43472582175275
(0, 0.0)
(0, 0.0)
    Residual: 0.0 (0, 0.0)


In [57]:
du.vector.size

6198

In [76]:
du.x.array.size

6198

In [66]:
V.dofmap.

<bound method DofMap.cell_dofs of <dolfinx.fem.dofmap.DofMap object at 0x7f91b4fdb9d0>>

In [52]:
import dolfinx

In [53]:
dolfinx.mesh.

_Feature((3, 7, 0, 'beta', 1), (3, 10, 0, 'alpha', 0), 16777216)

In [70]:
1478*2

2956

In [75]:
V.dofmap.index_map.size_global

3099

In [73]:
num_dofs_local = (V.dofmap.index_map.size_local) * V.dofmap.index_map_bs
num_dofs_global = V.dofmap.index_map.size_global * V.dofmap.index_map_bs

print(f"Number of dofs (owned) by rank : {num_dofs_local}")
# if COMM_WORLD.rank ==0:
print(f"Number of dofs global: {num_dofs_global}")

Number of dofs (owned) by rank : 6198
Number of dofs global: 6198
